目前为止，我们只是使用了 TensorFlow 的高级 API —— tf.keras，它的功能很强大：搭建了各种神经网络架构，包括回归、分类网络、Wide & Deep 网络、自归一化网络，使用了各种方法，包括批归一化、丢弃和学习率调度。事实上，你在实际案例中 95% 碰到的情况只需要tf.keras就足够了（和tf.data，见第 13 章）。现在来深入学习 TensorFlow 的低级 Python API。当你需要实现自定义损失函数、自定义标准、层、模型、初始化器、正则器、权重约束时，就需要低级 API 了。甚至有时需要全面控制训练过程，例如使用特殊变换或对约束梯度时。这一章就会讨论这些问题，还会学习如何使用 TensorFlow 的自动图生成特征提升自定义模型和训练算法。首先，先来快速学习下 TensorFlow。

### 1-TensorFlow 速览
TensorFlow 是一个强大的数值计算库，特别适合做和微调大规模机器学习（但也可以用来做其它的重型计算）。TensorFlow 是谷歌大脑团队开发的，支持了谷歌的许多大规模服务，包括谷歌云对话、谷歌图片和谷歌搜索。TensorFlow 是 2015 年 11 月开源的，（按文章引用、公司采用、GitHub 星数）是目前最流行的深度学习库。无数的项目是用 TensorFlow 来做各种机器学习任务，包括图片分类、自然语言处理、推荐系统和时间序列预测。TensorFlow 提供的功能如下：

-TensorFlow 的核心与 NumPy 很像，但 TensorFlow 支持 GPU；

-TensorFlow 支持（多设备和服务器）分布式计算；

-TensorFlow 使用了即时 JIT 编译器对计算速度和内存使用优化。编译器的工作是从 Python 函数提取出计算图，然后对计算图优化（比如剪切无用的节点），最后高效运行（比如自动并行运行独立任务）；

-计算图可以导出为迁移形式，因此可以在一个环境中训练一个 TensorFlow 模型（比如使用 Python 或 Linux），然后在另一个环境中运行（比如在安卓设备上用 Java 运行）；

-TensorFlow 实现了自动微分，并提供了一些高效的优化器，比如 RMSProp 和 NAdam，因此可以容易的最小化各种损失函数。

基于上面这些特点，TensorFlow 还提供了许多其他功能：最重要的是tf.keras，还有数据加载和预处理操作（tf.data，tf.io等等），图片处理操作（tf.image），信号处理操作（tf.signal），等等（图 12-1 总结了 TensorFlow 的 Python API）<br>
<img src='https://hands1ml.apachecn.org/docs/img/3d01200878f6c6d7033359da8291d199.png'><br>
TensorFlow 的低级操作都是用高效的 C++ 实现的。许多操作有多个实现，称为核：每个核对应一个具体的设备型号，比如 CPU、GPU，甚至 TPU（张量处理单元）。GPU 通过将任务分成小块，在多个 GPU 线程中并行运行，可以极大提高提高计算的速度。TPU 更快：TPU 是自定义的 ASIC 芯片，专门用来做深度学习运算的（第 19 章会讨论适合使用 GPU 和 TPU）。

TensorFlow 的架构见图 12-2。大多数时候你的代码使用高级 API 就够了（特别是tf.keras和tf.data），但如果需要更大的灵活性，就需要使用低级 Python API，来直接处理张量。TensorFlow 也支持其它语言的 API。任何情况下，甚至是跨设备和机器的情况下，TensorFlow 的执行引擎都会负责高效运行。<br>
<img src='https://hands1ml.apachecn.org/docs/img/ed0c3d9af331e1630b0d6051a6899d01.png'><br>
TensorFlow 不仅可以运行在 Windows、Linux 和 macOS 上，也可以运行在移动设备上（使用 TensorFlow Lite），包括 iOS 和安卓（见第 19 章）。如果不想使用 Python API，还可以使用 C++、Java、Go 和 Swift 的 API。甚至还有 JavaScript 的实现 TensorFlow.js，它可以直接在浏览器中运行。

TensorFlow 不只有这些库。TensorFlow 处于一套可扩展的生态系统库的核心位置。首先，TensorBoard 可以用来可视化。其次，TensorFlow Extended（TFX），是谷歌推出的用来生产化的库，包括：数据确认、预处理、模型分析和服务（使用 TF Serving，见第 19 章）。谷歌的 TensorFlow Hub 上可以方便下载和复用预训练好的神经网络。你还可以从 TensorFlow 的 model garden 获取许多神经网络架构，其中一些是预训练好的。TensorFlow Resources 和这个页面上有更多的资源。你可以在 GitHub 上找到数百个 TensorFlow 项目，无论干什么都可以方便地找到现成的代码。

最后，TensorFlow 有一支热忱满满的开发者团队，也有庞大的社区。要是想问技术问题，可以去这里 ，问题上打上 tensorflow 和 python 标签。还可以在 GitHub 上提 bug 和新功能。一般的讨论可以去谷歌群组。

默认情况下，Keras 使用均匀分布的 Glorot 初始化函数。创建层时，可以通过设置kernel_initializer="he_uniform"或kernel_initializer="he_normal"变更为 He 初始化，如下所示：

In [1]:
from tensorflow import keras

keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal") 
# 如果想让均匀分布的 He 初始化是基于fan[avg]而不是fan[in]，可以使用 VarianceScaling 初始化器：
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
                                                 distribution='uniform')
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init) 

非饱和激活函数(ReLU)>饱和激活函数

Glorot 和 Bengio 在 2010 年的论文中的一个见解是，消失/爆炸的梯度问题部分是由于激活函数的选择不好造成的。 在那之前，大多数人都认为，如果大自然选择在生物神经元中使用 sigmoid 激活函数，它们必定是一个很好的选择。 但事实证明，其他激活函数在深度神经网络中表现得更好，特别是 ReLU 激活函数，主要是因为它对正值不会饱和（也因为它的计算速度很快）。<br>


#### ReLU变体

但是，ReLU 激活功能并不完美。 它有一个被称为 “ReLU 死区” 的问题：在训练过程中，一些神经元会“死亡”，即它们停止输出 0 以外的任何东西。在某些情况下，你可能会发现你网络的一半神经元已经死亡，特别是使用大学习率时。 在训练期间，如果神经元的权重得到更新，使得神经元输入的加权和为负，则它将开始输出 0 。当这种情况发生时，由于当输入为负时，ReLU 函数的梯度为 0，神经元就只能输出 0 了。

1-leaky ReLU

为了解决这个问题，你可能需要使用 ReLU 函数的一个变体，比如 leaky ReLU。这个函数定义为LeakyReLU[α](z)= max(αz, z)（见图 11-2）。超参数α定义了函数“泄露”的程度：它是z < 0时函数的斜率，通常设置为 0.01。这个小斜率保证 leaky ReLU 永不死亡；他们可能会长期昏迷，但他们有机会最终醒来。2015 年的一篇论文比较了几种 ReLU 激活功能的变体，其中一个结论是 leaky Relu 总是优于严格的 ReLU 激活函数。事实上，设定α= 0.2（大的泄露）似乎比α= 0.01（小的泄露）有更好的性能。这篇论文还评估了随机化 leaky ReLU（RReLU），其中α在训练期间在给定范围内随机，并在测试期间固定为平均值。它表现相当好，似乎是一个正则项（减少训练集的过拟合风险）。最后，文章还评估了参数化的 leaky ReLU（PReLU），其中α被授权在训练期间参与学习（而不是作为超参数，α变成可以像任何其他参数一样被反向传播修改的参数）。据报道，PReLU 在大型图像数据集上的表现强于 ReLU，但是对于较小的数据集，其具有过度拟合训练集的风险。<br>
<img src='https://hands1ml.apachecn.org/docs/img/9679c67f28c8b2370252dc5cc7a45e61.png'><br>
2-ELU

最后，Djork-Arné Clevert 等人在 2015 年的一篇论文中提出了一种称为指数线性单元（exponential linear unit，ELU）的新激活函数，在他们的实验中，ELU 的表现优于所有 ReLU 变体：训练时间减少，神经网络在测试集上表现的更好。 如图 11-3 所示，公式 11-2 给出了它的定义。<br>
<img src='https://hands1ml.apachecn.org/docs/img/e40be2d92991f7daafc0b96699a9208e.png'><br>
ELU 看起来很像 ReLU 函数，但有一些区别，主要区别在于：<br>
-它在z < 0时取负值，这使得该单元的平均输出接近于 0。这有助于减轻梯度消失问题。 超参数α定义为当z是一个大的负数时，ELU 函数接近的值。它通常设置为 1，但是如果你愿意，你可以像调整其他超参数一样调整它。<br>
-它对z < 0有一个非零的梯度，避免了神经元死亡的问题。<br>
-如果α等于 1，则函数在任何地方都是平滑的，包括z = 0附近，这有助于加速梯度下降，因为它不会在z = 0附近回弹。<br>
ELU 激活函数的主要缺点是计算速度慢于 ReLU 及其变体（由于使用指数函数），但是在训练过程中，这是通过更快的收敛速度来补偿的。 然而，在测试时间，ELU 网络将比 ReLU 网络慢。<br>
<img src='https://hands1ml.apachecn.org/docs/img/a83b3a515cc64f90660ae88f90fa182c.png'><br>
3-Scaled ELU（SELU）

2017 年的一篇文章中，Günter Klambauer 等人介绍了一种 Scaled ELU（SELU）激活函数：正如它的名字所示，它是 ELU 的伸缩变体。作者证明，只要神经网络中都是紧密层，并且所有隐藏层都是用的 SELU 激活函数，则这个网络是自归一的：训练过程中，每层输出的平均值是 0，标准差是 1，这样就解决了梯度消失爆炸问题。对于全紧密层的网络（尤其是很深的），SELU 的效果常常优于其他激活函数。但是自归一是需要条件的（数学论证见论文）：<br>
-输入特征必须是标准的（平均值是 0，标准差是 1）；<br>
-每个隐藏层的权重必须是 LeCun 正态初始化的。在 Keras 中，要设置kernel_initializer="lecun_normal"；<br>
-网络架构必须是顺序的。但是，如果要在非顺序网络（比如 RNN）或有跳连接的网络（跳过层的连接，比如 Wide&Deep）中使用 SELU，就不能保证是自归一的，所以 SELU 就不会比其它激活函数更优；<br>
-这篇论文只是说如果所有层都是紧密层才保证自归一，但有些研究者发现 SELU 激活函数也可以提高卷积神经网络的性能。

提示：那么深层神经网络的隐藏层应该使用哪个激活函数呢？ 虽然可能会有所不同，一般来说 SELU > ELU > leaky ReLU（及其变体）> ReLU > tanh > sigmoid。 如果网络架构不能保证自归一，则 ELU 可能比 SELU 的性能更好（因为 SELU 在z=0时不是平滑的）。如果关心运行延迟，则 leaky ReLU 更好。 如果你不想多调整另一个超参数，你可以使用前面提到的默认的α值（leaky ReLU 为 0.3）。 如果有充足的时间和计算能力，可以使用交叉验证来评估其他激活函数，如果神经网络过拟合，则使用 RReLU; 如果您拥有庞大的训练数据集，则为 PReLU。但是，因为 ReLU 是目前应用最广的激活函数，许多库和硬件加速器都使用了针对 ReLU 的优化，如果速度是首要的，ReLU 可能仍然是首选。

In [4]:
# 要使用 leaky ReLU，需要创建一个LeakyReLU层，并将它加到需要追加的层后面：
# 对于 PReLU，用PReLU()替换LeakyRelu(alpha=0.2)。
# 目前还没有 RReLU 的 Keras 官方实现，但很容易自己实现（方法见第 12 章的练习）。
model1 = keras.models.Sequential([
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
]) 

In [5]:
# 对于 SELU，当创建层时设置activation="selu"，kernel_initializer="lecun_normal"：
layer1 = keras.layers.Dense(10, activation="selu",
                           kernel_initializer="lecun_normal") 

#### 批归一化（Batch Normalization）
尽管使用 He 初始化和 ELU（或任何 ReLU 变体）可以显著减少训练开始阶段的梯度消失/爆炸问题，但不能保证在训练期间问题不会再次出现。<br>
在 2015 年的一篇论文中，Sergey Ioffe 和 Christian Szegedy 提出了一种称为批归一化（Batch Normalization，BN）的方法来解决梯度消失/爆炸问题。该方法包括在每层的激活函数之前或之后在模型中添加操作。操作就是将输入平均值变为 0，方差变为 1，然后用两个新参数，一个做缩放，一个做偏移。换句话说，这个操作可以让模型学习到每层输入值的最佳缩放值和平均值。大大多数情况下，如果模型的第一层使用了 BN 层，则不用标准化训练集（比如使用StandardScaler）；BN 层做了标准化工作（虽然是近似的，每次每次只处理一个批次，但能做缩放和平移）。<br>
为了对输入进行零居中（平均值是 0）和归一化，算法需要估计输入的均值和标准差。 它通过评估当前小批量输入的均值和标准差（因此命名为“批归一化”）来实现。 整个操作在公式 11-3 中。<br>
<img src='https://hands1ml.apachecn.org/docs/img/aa3b950c85c9c9474f98029f6a560299.png'><br>
其中，

μ[B]是整个小批量B的均值向量

σ[B]是输入标准差向量，也是根据整个小批量估算的。

m[B]是小批量中的实例数量。

X_hat^(j)是以为零中心和标准化的实例i的输入向量。

γ是层的缩放参数的向量（每个输入一个缩放参数）。

⊗表示元素级别的相乘（每个输入乘以对应的缩放参数）

β是层的偏移参数（偏移量）向量（每个输入一个偏移参数）

ϵ是一个很小的数字，以避免被零除（通常为10^-5）。 这被称为平滑项（拉布拉斯平滑，Laplace Smoothing）。

z^(i)是 BN 操作的输出：它是输入的缩放和移位版本。

在训练时，BN 将输入标准化，然后做了缩放和平移。测试时又如何呢？因为需要对实例而不是批次实例做预测，所以就不能计算每个输入的平均和标准差。另外，即使有批量实例，批量也可能太小，或者实例并不是独立同分布的，所以在批量上计算是不可靠的。一种解决方法是等到训练结束，用模型再运行一次训练集，算出每个 BN 层的平均值和标准差。然后就可以用这些数据做预测，而不是批输入的平均值和标准差。但是，大部分批归一化实现是通过层输入的平均值和标准差的移动平均值来计算的。这也是 Keras 在BatchNormalization中使用的方法。总的来说，每个批归一化的层都通过指数移动平均学习了四个参数：γ（输出缩放向量），β（输出偏移向量），μ（最终输入平均值向量）和σ（最终输入标准差向量）。μ和σ都是在训练过程中计算的，但只在训练后使用（用于替换公式 11-3 中批输入平均和标准差）。<br>
Ioffe 和 Szegedy 证明，批归一化大大改善了他们试验的所有深度神经网络，极大提高了 ImageNet 分类的效果（ImageNet 是一个图片分类数据集，用于评估计算机视觉系统）。梯度消失问题大大减少了，他们可以使用饱和激活函数，如 tanh 甚至逻辑激活函数。网络对权重初始化也不那么敏感。他们能够使用更大的学习率，显著加快了学习过程。具体地，他们指出，“应用于最先进的图像分类模型，批标准减少了 14 倍的训练步骤实现了相同的精度，以显著的优势击败了原始模型。[...] 使用批量标准化的网络集合，我们改进了 ImageNet 分类上的最佳公布结果：达到 4.9% 的前 5 个验证错误（和 4.8% 的测试错误），超出了人类评估者的准确性。批量标准化也像一个正则化项一样，减少了对其他正则化技术的需求（如本章稍后描述的丢弃）.<br>
然而，批量标准化的确会增加模型的复杂性（尽管它不需要对输入数据进行标准化，因为第一个隐藏层会照顾到这一点，只要它是批量标准化的）。 此外，还存在运行时间的损失：由于每层所需的额外计算，神经网络的预测速度较慢。 但是，可以在训练之后，处理在 BN 层的前一层，就可以加快速度。方法是更新前一层的权重和偏置项，使其直接输出合适的缩放值和偏移值。例如，如果前一层计算的是XW + b，BN 层计算的是γ⊗(XW + b – μ)/σ + β（忽略了分母中的平滑项ε）。如果定义W′ = γ⊗W/σ和b′ = γ⊗(b – μ)/σ + β，公式就能简化为XW′ + b′。因此如果替换前一层的权重和偏置项（W和b）为W'和b'，就可以不用 BN 层了（TFLite 的优化器就干了这件事，见第 19 章）。<br>
注意：你可能会发现，训练相当缓慢，这是因为每个周期都因为使用 BN 而延长了时间。但是有了 BN，收敛的速度更快，需要的周期数更少。综合来看，需要的总时长变短了。

使用 Keras 实现批归一化

和 Keras 大部分功能一样，实现批归一化既简单又直观。只要每个隐藏层的激活函数前面或后面添加一个BatchNormalization层就行，也可以将 BN 层作为模型的第一层。例如，这个模型在每个隐藏层的后面使用了 BN，第一层也用了 BN（在打平输入之后）：

In [6]:
model2 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
]) 

In [7]:
# 这样就成了！在这个只有两个隐藏层的例子中，BN 的作用不会那么大，但对于更深的网络，作用就特别大。
# 打印一下模型的摘要：
model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_6 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

可以看到每个 BN 层添加了四个参数：γ、 β、 μ 和 σ（例如，第一个 BN 层添加了 3136 个参数，即4 × 784）。后两个参数μ 和 σ是移动平均，不受反向传播影响，Keras 称其“不可训练”（如果将 BN 的总参数3,136 + 1,200 + 400除以 2，得到 2368，就是模型中总的不可训练的参数量）。

In [8]:
# 看下第一个 BN 层的参数。两个参数是可训练的（通过反向传播），两个不可训练：
[(var.name, var.trainable) for var in model2.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [10]:
# 当在 Keras 中创建一个 BN 层时，训练过程中，还会创建两个 Keras 在迭代时的操作。
# 该操作会更新移动平均值。因为后端使用的是 TensorFlow，这些操作就是 TensorFlow 操作
model2.layers[1].updates

C:\Users\86182\AppData\Local\Temp\ipykernel_13588\2131884122.py:3: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  model2.layers[1].updates


[]

BN 的论文作者建议在激活函数之前使用 BN 层，而不是像前面的例子添加到后面。到底是前面还是后面好存在争议，取决于具体的任务 —— 你最好在数据集上试验一下哪种选择好。要在激活函数前添加 BN 层，必须将激活函数从隐藏层拿出来，单独做成一层。另外，因为 BN 层对每个输入有一个偏移参数，可以将前一层的偏置项去掉（设置use_bias=False）：

In [11]:
model3 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),

    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    
    keras.layers.Dense(10, activation="softmax")
]) 

BatchNormalization类可供调节的参数不多。默认值通常就可以，但有时需要调节momentum，这个超参数是BatchNormalization在更新指数移动平均时使用的。给定一个新值v（即，一个当前批次的输入平均或标准差新向量），BN 层使用下面的等式更新平均V_hat: momentum的最优值通常接近于 1：比如，0.9、0.99、0.999（大数据的 9 更多，小数据集的 9 少）。<br>
<img src='https://hands1ml.apachecn.org/docs/img/c7482a9798005dd55876cc837c6919f9.png'><br>
另一个重要的超参数是axis：它确定了在哪个轴上归一。默认是 -1，即归一化最后一个轴（使用其它轴的平均值和标准差）。当输入是 2D 时（即批的形状是[batch size,features]），也就是说每个输入特征都会根据批次全部实例的平均值和标准差做归一。例如，前面例子的第一个 BN 层会分别对 784 个输入特征的每个特征做归一化（还有缩放和偏移）；因此，BN 层会计算 28 个平均值和 28 个标准差（每列 1 个值，根据每行的所有实例计算），用同样的平均值和标准差归一化给定列的所有像素。还会有 28 个缩放值和 28 个偏移值。如果仍想对 784 个像素独立处理，要设置axis=[1, 2]。<br>
在训练和训练之后，BN 层不会做同样的计算：BN 会使用训练中的批次数据和训练后的最终数据（即移动平均值的最终值）。看看源码中是如何实现的：<br>
call()方法具体实现了方法，它有一个参数training，默认是None，但fit()方法在训练中将其设为 1。如果你需要写一个自定义层，要求自定义层在训练和测试中的功能不同，就可以在call()方法中添加一个参数training，用这个参数决定该计算什么（第 12 张会讨论自定义层）。<br>

In [12]:
class BatchNormalization(keras.layers.Layer):
    [...]
    def call(self, inputs, training=None):
        [...] 

BatchNormalization已经成为了深度神经网络中最常使用的层，以至于计算图中经常省略，默认嘉定在每个层后面加一个 BN 层。但是 Hongyi Zhang 的一篇文章可能改变了这种做法：通过使用一个新的fixed-update（fixup）权重初始化方法，作者没有使用 BN，训练了一个非常深的神经网络（多达 10000 层），在复杂图片分类任务上表现惊艳。但这个结论很新，最好还是再等一等，现在还是使用批归一化。

### 1-像 NumPy 一样使用 TensorFlow
TensorFlow 的 API 是围绕张量（tensor）展开的，从一个操作流动（flow）到另一个操作，所以名字叫做 TensorFlow。张量通常是一个多维数组（就像 NumPy 的ndarray），但也可以是标量（即简单值，比如 42）。张量对于自定义的损失函数、标准、层等等非常重要，接下来学习如何创建和操作张量。

#### 张量和运算
